Inspiration for Neural Net: https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html

# Data Pre-Processing

In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pitcher
from random import shuffle
torch.manual_seed(1)

In [5]:
player = 'verlander'
data = pd.read_csv('Data/raw_data/'+player+'.csv')
data = pitcher.clean_data(data)
ABs = pitcher.get_abs(data)
reps = pitcher.get_reps(ABs)
reps = pitcher.drop_nas(reps)

In [6]:
#get train, validate, and test sets (60-20-20)
shuffle(reps)
cutoff1 = int(len(reps)*0.6)
cutoff2 = cutoff1 + int(len(reps)*.2)
train = reps[1:cutoff1]
validate = reps[cutoff1:cutoff2]
test = reps[cutoff2:]


#get train, validate, and test batches
batch_size = 100
train_batches = pitcher.get_batches(train,batch_size)
test_batches = pitcher.get_batches(test,batch_size)
validate_batches = pitcher.get_batches(validate,batch_size)

# Model Construction

In [42]:
#change this for different pitcher
'''tag_to_ix = {'CU':0,
             'FF':1,
             'SL':2,
             'CH':3}'''
tag_to_ix = {'NF':0,
             'FF':1}

# -- Input Dimensions -- DO NOT CHANGE
PREV_PITCH_DIM = 25
NUM_PREV_PITCHES = 3
GAME_STATE_DIM = 15 
GAME_OUT_DIM = 5

# -- Hyperparameters -- DO CHANGE 
HIDDEN_DIM = 20
OUT_DIM = 10
lstm_layers = 1
learning_rate = 0.0001

if torch.cuda.is_available():
    torch.set_default_tensor_type(torch.cuda.FloatTensor)
    
class PitchPredict(nn.Module):
    def __init__(self, prev_pitch_dim, hidden_dim, num_prev_pitches,out_dim, game_state_dim, game_out_dim, num_ptypes):
        super(PitchPredict, self).__init__()
        #get constants
        self.hidden_dim = hidden_dim
        self.prev_pitch_dim = prev_pitch_dim
        self.num_prev_pitches = num_prev_pitches
        self.out_dim = out_dim
        self.game_state_dim = game_state_dim
        self.game_out_dim = game_out_dim
        
        ####Define Layers####
        
        ########################## LSTM for past five pitches#######################################
        # The LSTM takes previous pitch vectors as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(prev_pitch_dim, hidden_dim,num_layers=lstm_layers,batch_first=True)#FIDDLE WITH num_layers

        # The linear layer that maps from hidden state space to a vector
        # with dimensionality OUT_DIM
        self.hidden2out = nn.Linear(hidden_dim, out_dim)
        
        ############## FULLY CONNECTED LAYERS for LSTM OUTPUT + game_state vector #################
        
        # layer to map the game state vector to a different dimension
        self.l1 = nn.Linear(self.game_state_dim, self.game_out_dim)  
        
        # This Fully connected layer maps from the output of the final hidden layer output from the LSTM,
        # dimension = OUT_DIM, with the game state vector, dimension = GAME_STATE_DIM
        # to a vector of length of the number of ptypes to pass through softmax for probabilities
        self.fc1 = nn.Linear((self.out_dim + self.game_out_dim), num_ptypes)

                             
    def forward(self, rep):
        past_pitches,game_state = rep
        lstm_out, _ = self.lstm(past_pitches.view(batch_size,self.num_prev_pitches, -1))
        learned_rep = self.hidden2out(lstm_out.view(batch_size,self.num_prev_pitches, -1))
        #encoding = F.relu(learned_rep[:,self.num_prev_pitches - 1:,:])
        game_rep = game_state.view(batch_size,self.game_state_dim)
        game_rep = self.l1(game_state.view(batch_size,self.game_state_dim))
        game_rep = F.relu(game_rep)
        encoding = learned_rep[:,self.num_prev_pitches - 1:,:]
        fc_in = torch.cat((encoding.view(batch_size,self.out_dim),game_rep.view(batch_size,self.game_out_dim)),dim=1)
        fc = self.fc1(fc_in.view(batch_size,self.game_out_dim+self.out_dim))
        tag_scores = F.log_softmax(fc,dim=0)
        return tag_scores

# Testing

In [8]:
def test_accuracy(batches,model):
    length = len(batches)*len(batches[0][0])
    num_right = 0
    ch_count = 0
    predict_ch = 0
    cu_count = 0
    predict_cu = 0
    sl_count = 0
    predict_sl = 0
    ff_count = 0
    predict_ff = 0
    ff_right = 0
    ch_right = 0
    sl_right = 0
    cu_right = 0
    for batch in batches:
        with torch.no_grad():
            prev_pitches,pre_pitch, ptypes = batch
            prevs_in = torch.tensor(prev_pitches, dtype=torch.float)
            rep_in = torch.tensor(pre_pitch, dtype=torch.float)
            #targets = tag_to_ix[ptypes]
            if len(ptypes) > 1:
                targets = [tag_to_ix[ptype] for ptype in ptypes]
            else:
                targets = tag_to_ix[ptypes[0]]
            if torch.cuda.is_available():
                tag_scores = model((prevs_in,rep_in)).cuda()
            else:
                tag_scores = model((prevs_in,rep_in))
            #print(tag_scores)
            preds = [tag_score.max(0) for tag_score in tag_scores]
            #print(index.item())
            if len(ptypes) > 1:
                for pred,target in zip(preds,targets):
                    _,index = pred
                    if target == 1:
                        ff_count += 1
                    if index.item() == 1:
                        predict_ff += 1
                    if target == 1 and index.item() == 1:
                        ff_right += 1
                    if target == 0:
                        cu_count += 1
                    if index.item() == 0:
                        predict_cu += 1
                    if target == 0 and index.item() == 0:
                        cu_right += 1
                    if target == 2:
                        sl_count += 1
                    if index.item() == 2:
                        predict_sl += 1
                    if target == 2 and index.item() == 2:
                        sl_right += 1
                    if target == 3:
                        ch_count += 1
                    if index.item() == 3:
                        predict_ch += 1
                    if target == 3 and index.item() == 3:
                        ch_right += 1
                    if index.item() == target:
                        num_right += 1
            else:
                _,index = preds[0]
                if targets == 1:
                    ff_count += 1
                if index.item() == 1:
                    predict_ff += 1
                if targets == 1 and index.item() == 1:
                    ff_right += 1
                if targets == 0:
                    cu_count += 1
                if index.item() == 0:
                    predict_cu += 1
                if targets == 0 and index.item() == 0:
                    cu_right += 1
                if targets == 2:
                    sl_count += 1
                if index.item() == 2:
                    predict_sl += 1
                if targets == 2 and index.item() == 2:
                    sl_right += 1
                if targets == 3:
                    ch_count += 1
                if index.item() == 3:
                    predict_ch += 1
                if targets == 3 and index.item() == 3:
                    ch_right += 1
                if index.item() == targets:
                    num_right += 1
    ff_rate = ff_count/length
    pred_ff_rate = predict_ff/length
    ff_acc = ff_right/ff_count
    cu_rate = cu_count/length
    pred_cu_rate = predict_cu/length
    cu_acc = cu_right/cu_count
    #sl_rate = sl_count/length
    #pred_sl_rate = predict_sl/length
    #sl_acc = sl_right/sl_count
    #ch_rate = ch_count/length
    #pred_ch_rate = predict_ch/length
    #ch_acc = ch_right/ch_count
    accuracy = num_right/length
    print("______________________________________")
    print("Non-Fastball rate:",cu_rate)
    print("Predicted Non-Fastball rate:",pred_cu_rate)
    print("Non-Fastball accuracy:",cu_acc)
    print("______________________________________")
    print("Fastball rate:",ff_rate)
    print("Predicted Fastball rate:",pred_ff_rate)
    print("Fastball accuracy:",ff_acc)
    #print("______________________________________")
    #print("Changeup rate:",ch_rate)
    #print("Predicted changeup rate:",pred_ch_rate)
    #print("changeup accuracy",ch_acc)
    #print("______________________________________")
    #print("slider rate:",sl_rate)
    #print("Predicted slider rate:",pred_sl_rate)
    #print("slider accuracy:",sl_acc)
    print("______________________________________")
    print("Accuracy:",accuracy)
    print("Accuracy above naive guess:",accuracy - ff_rate)
    print("______________________________________")

# Training

In [43]:
model = PitchPredict(PREV_PITCH_DIM, HIDDEN_DIM, NUM_PREV_PITCHES, OUT_DIM, GAME_STATE_DIM, GAME_OUT_DIM, len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)#FIDDLE WITH LEARNING RATE (lr)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    print("***************Pre-Training Accuracy*****************")
    test_accuracy(train_batches,model)
print("**************************Training*****************************")
for epoch in range(25):
    #shuffle(train_batches)
    for batch in train_batches:
        prev_pitches,pre_pitch, ptypes = batch
        
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        #get input tensors ready
        prevs_in = torch.tensor(prev_pitches, dtype=torch.float)
        game_state_in = torch.tensor(pre_pitch, dtype=torch.float)
        
        #get target value
        #target = tag_to_ix[ptype]
        target = [ tag_to_ix[ptype] for ptype in ptypes]

        # Step 3. Run our forward pass.
        if torch.cuda.is_available():
            tag_scores = model((prevs_in,game_state_in)).cuda()
        else:
            tag_scores = model((prevs_in,game_state_in))
               
        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores.view(batch_size,-1), torch.tensor(target,dtype=torch.long))
        loss.backward()
        optimizer.step()
        
        preds = [tag_score.max(0) for tag_score in tag_scores]

                
    #display post-epoch results
    print('epoch:',epoch+1,"loss:",loss.item())

# See what the scores are after training
with torch.no_grad():
    print("****************Post-Training Accuracy********************")
    test_accuracy(train_batches,model.eval())
    print("**********************************************************")

***************Pre-Training Accuracy*****************
______________________________________
Non-Fastball rate: 0.4197457627118644
Predicted Non-Fastball rate: 0.48491525423728815
Non-Fastball accuracy: 0.49030890369473046
______________________________________
Fastball rate: 0.5802542372881356
Predicted Fastball rate: 0.5150847457627119
Fastball accuracy: 0.5189864174090842
______________________________________
Accuracy: 0.5069491525423728
Accuracy above naive guess: -0.0733050847457628
______________________________________
**************************Training*****************************
epoch: 1 loss: 4.60304069519043
epoch: 2 loss: 4.5981974601745605
epoch: 3 loss: 4.593774795532227
epoch: 4 loss: 4.589673042297363
epoch: 5 loss: 4.585574626922607
epoch: 6 loss: 4.581826686859131
epoch: 7 loss: 4.578592300415039
epoch: 8 loss: 4.575615882873535
epoch: 9 loss: 4.573040962219238
epoch: 10 loss: 4.570858001708984
epoch: 11 loss: 4.569002151489258
epoch: 12 loss: 4.567403793334961
epoc

# Results

In [44]:
with torch.no_grad():
    print("****************Validation Accuracy********************")
    test_accuracy(validate_batches,model)
    print("**********************************************************")

****************Validation Accuracy********************
______________________________________
Non-Fastball rate: 0.41384615384615386
Predicted Non-Fastball rate: 0.5188461538461538
Non-Fastball accuracy: 0.661090458488228
______________________________________
Fastball rate: 0.5861538461538461
Predicted Fastball rate: 0.48115384615384615
Fastball accuracy: 0.581583552055993
______________________________________
Accuracy: 0.6144871794871795
Accuracy above naive guess: 0.02833333333333332
______________________________________
**********************************************************


In [103]:
with torch.no_grad():
    print("****************Test Accuracy********************")
    test_accuracy(test_batches,model)
    print("**********************************************************")

****************Test Accuracy********************
______________________________________
Non-Fastball rate: 0.41363057324840763
Predicted Non-Fastball rate: 0.47388535031847134
Non-Fastball accuracy: 0.6214967662457653
______________________________________
Fastball rate: 0.5863694267515923
Predicted Fastball rate: 0.5261146496815287
Fastball accuracy: 0.6302411470779926
______________________________________
Accuracy: 0.6266242038216561
Accuracy above naive guess: 0.04025477707006375
______________________________________
**********************************************************


### high score using balanced set (25% FF)
- **TRAINING ACCURACY HIGH SCORE:** 38 (39% FF)

- **TEST ACCURACY HIGH SCORE:** 43.7 (37% FF) -14%- naive 


- ****HYPERPARAMETERS:****

    - LSTM num_layers: 1
    
    - OUT_SIZE: 15
    
    - HIDDEN_DIMENSION: 10
    
    - EPOCHS: 5
   
    - LEARNING RATE: 0.001
    
    - LOSS FUNCTION: NLLLOSS
    
    - OPTIMIZER: ADAM

### high score using unbalanced set (50+% FF)
- **TRAINING ACCURACY HIGH SCORE:** 57.8 (91% FF)

- **TEST ACCURACY HIGH SCORE:** 60.86 (88% FF) 2.5%+ naive 


- ****HYPERPARAMETERS:****

    - LSTM num_layers: 1
    
    - OUT_SIZE: 15
    
    - HIDDEN_DIMENSION: 10
    
    - EPOCHS: 20
   
    - LEARNING RATE: 0.001
    
    - LOSS FUNCTION: NLLLOSS
    
    - OPTIMIZER: ADAM